# The Database Interface Classes

For each API endpoint exposed in the Django app, there is a corresponding class that
provide methods to execute CRUD operations asynchronously.

There are two types of API endpoints -- those that contain only records data, and 
those that store both records and pointers to files.



## Connecting to the Database

The database currently runs on HTCF service partition. This is a single
node with 8 CPU and 30 GB that is meant for long running low resource jobs.
The components that need to run are a postgres database, a redis instance and the
django app. As long as these components are running on the service partition, 
you can connect via an ssh tunnel with:

```bash
ssh username@login.htcf.wustl.edu -N -L 8001:n240:8000
```

where the `8001:n240:8000` takes the form of `local_port:cluster_node:app_port`. The
django app will always be served on port `8000`, and `n240` is the only service
partition node. You may choose a different local port.

If you do this and cannot connect, let me know and I'll check the status of the jobs 
on the cluster.

### Database username and password

Once you have a tunnel, you can access the database frontend at `127.0.0.1:8001`
(or a different local port, if you changed that number). If you haven't already
signed up, you'll need to click the 'sign up' button and follow
the instructions. The e-mail server is not hooked up at the moment, so when it says
"see the e-mail", send a slack message and let me know. I'll give you a link to
complete the sign up process. After that, you can just use the "sign in" button.

For computational tasks, including generating rank response data, celery workers must
be launched on the HTCF general partition. There is currently a script that is meant to
monitor the redis queue and launch/kill these workers automatically, but this
functionality is new and largely untested. You can monitor the workers/tasks if you
create another tunnel with:

```bash
ssh username@login.htcf.wustl.edu -N -L 8002:n240:5555
```
You'd access this dashboard at `127.0.0.1:5555`

The login is currently:

```raw
username: "flower"
password: "daisy"
```
(yes, really -- the security comes from the fact that you need to login with HTCF)

### Configuring the Database Interface Classes

The database classes expect the following environmental variables to be set. 

```raw
BASE_URL='http://127.0.0.1:8001'
TOKEN='<your token>'
BINDING_URL='http://127.0.0.1:8001/api/binding'
BINDINGMANUALQC_URL='http://127.0.0.1:8001/api/bindingmanualqc'
CALLINGCARDSBACKGROUND_URL='http://127.0.0.1:8001/api/callingcardsbackground'
DATASOURCE_URL='http://127.0.0.1:8001/api/datasource'
EXPRESSION_URL='http://127.0.0.1:8001/api/expression'
EXPRESSIONMANUALQC_URL='http://127.0.0.1:8001/api/expressionmanualqc'
FILEFORMAT_URL='http://127.0.0.1:8001/api/fileformat'
GENOMICFEATURE_URL='http://127.0.0.1:8001/api/genomicfeature'
PROMOTERSET_URL='http://127.0.0.1:8001/api/promoterset'
PROMOTERSETSIG_URL='http://127.0.0.1:8001/api/promotersetsig'
REGULATOR_URL='http://127.0.0.1:8001/api/regulator'
```

This can be achieved in the package during development with a `.env` file at the
top most level of the package. The `.env` file is loaded in the package `__init__.py`.

If you are importing `yeastdnnexplorer` into a different environment, then you'll 
need to add the package `dotenv` and execute `load_dotenv(dotenv_path=env_path)`. If
the `.env` file is in the same `PWD` in which you execute that command, there is no
need to specify a path.

### Token Authentication

Once you have a username and password to the database, you can retrieve your token. 
Make sure that you put this token, at least, in a `.env` file, and make sure that 
`.env` file is in your `.gitignore`.

Alternatively, you could retrieve and store in memory the token at the beginning of 
each session -- this is more secure if you are not using a `.env` file.  

The `.env` file is already in the `yeastddnexplorer` `.gitignore`

```bash
curl -X 'POST' \
  'http://127.0.0.1:8001/auth-token/' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "username": "username",
  "password": "password"
}'
```

Or with python:

```python
import requests

url = "http://127.0.0.1:8001/auth-token/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
}
data = {
    "username": "username",
    "password": "password",
}

response = requests.post(url, json=data, headers=headers)
print(response.text)
```

## Using the Interface Classes

In [49]:
from yeastdnnexplorer.interface import *

## Records Only Endpoints

The records only endpoints are:

- BindingManualQC

- DataSource

- ExpressionManualQC

- FileFormat

- GenomicFeature

- PromoterSetSig

- Regulator

When the `read()` method is called on the corresponding API classes, a dataframe will
be returned in the response.

All of the `read()` methods, for both types of API endpoints, return the result of
a callable. By default, the callable returns a dictionary with two keys: `metadata` and
`data`. For response only tables, the `metadata` value will be the records from the
database as a pandas dataframe and the `data` will be `None.

### Example -- RegulatorAPI

In [50]:
regulator = RegulatorAPI()

result = await regulator.read()
result.get("metadata")

,id,uploader_id,upload_date,modifier_id,modified_date,genomicfeature_id,under_development,notes,regulator_locus_tag,regulator_symbol
0,6,1,2024-03-18,1,2024-03-18 18:33:43.005782+00:00,17,False,none,YAL059W,ECM1
1,7,1,2024-03-18,1,2024-03-18 18:33:44.106488+00:00,20,False,none,YAL056W,GPB2
2,8,1,2024-03-18,1,2024-03-18 18:33:44.605193+00:00,21,False,none,YAL055W,PEX22
3,9,1,2024-03-18,1,2024-03-18 18:33:44.891194+00:00,22,False,none,YAL054C,ACS1
4,10,1,2024-03-18,1,2024-03-18 18:33:45.111148+00:00,23,False,none,YAL053W,FLC2
...,...,...,...,...,...,...,...,...,...,...
1810,1816,1,2024-03-18,1,2024-03-18 23:13:22.054254+00:00,5258,False,none,YMR168C,CEP3
1811,1817,1,2024-03-18,1,2024-03-18 23:41:58.734300+00:00,5878,False,none,YNR054C,ESF2
1812,1818,1,2024-03-25,1,2024-03-25 20:04:21.181017+00:00,569,False,none,YBR267W,REI1
1813,1819,1,2024-03-25,1,2024-03-25 20:04:32.036007+00:00,1171,False,none,YDR081C,PDC2


## Record and File Endpoints

The record and file endpoints are the following:

- CallingCardsBackground

- Expression

- PromoterSet

- PromoterSetSig

- RankResponse *

The default `read()` method is the same as the Records only Endpoint API classes.
However, there is an additional argument, `retrieve_files` which if set to `True`
will retrieve the file for which each record provides metadata. The return value of
`read()` is again a callable, and by default the `data` key will store a dictionary
where the keys correspond to the `id` column in the `metadata`.


In [51]:
# First, retrieve only the records -- you'll want to filter these results down before
# retrieving the files most likely
pss_api = PromoterSetSigAPI()
result = await pss_api.read()
result.get("metadata")

,id,uploader_id,upload_date,modifier_id,modified_date,binding_id,promoter_id,background_id,fileformat_id,file
0,8419,1,2024-03-25,1,2024-03-25 20:04:11.646870+00:00,3011,NaN,NaN,1,promotersetsig/8419.csv.gz
1,8420,1,2024-03-25,1,2024-03-25 20:04:12.153311+00:00,3012,NaN,NaN,1,promotersetsig/8420.csv.gz
2,8421,1,2024-03-25,1,2024-03-25 20:04:12.508328+00:00,3013,NaN,NaN,1,promotersetsig/8421.csv.gz
3,8422,1,2024-03-25,1,2024-03-25 20:04:12.835947+00:00,3014,NaN,NaN,1,promotersetsig/8422.csv.gz
4,8423,1,2024-03-25,1,2024-03-25 20:04:13.162373+00:00,3015,NaN,NaN,1,promotersetsig/8423.csv.gz
...,...,...,...,...,...,...,...,...,...,...
2178,11091,1,2024-03-26,1,2024-03-26 14:30:28.156704+00:00,4481,4.0,6.0,5,promotersetsig/11091.csv.gz
2179,11092,1,2024-03-26,1,2024-03-26 14:30:28.218468+00:00,4480,4.0,6.0,5,promotersetsig/11092.csv.gz
2180,11093,1,2024-03-26,1,2024-03-26 14:30:28.310173+00:00,4482,4.0,6.0,5,promotersetsig/11093.csv.gz
2181,11094,1,2024-03-26,1,2024-03-26 14:30:28.695849+00:00,4483,4.0,6.0,5,promotersetsig/11094.csv.gz


## Filtering

All API classes have a `params` attribute which stores the filtering parameters
which will be applied to the HTTP requests.

In [52]:
pss_api.push_params({"regulator_symbol": "HAP5",
                     "workflow": "nf_core_callingcards_dev",
                     "data_usable": "pass"})

## Retrieving files from a Records and Files Object

To retrieve files from a Records and Files endpoint object, do the following:

In [53]:
# note that retrieve_files is set to True
result = await pss_api.read(retrieve_files = True)

# the metadata slot is the same as before
result.get("metadata")

,id,uploader_id,upload_date,modifier_id,modified_date,binding_id,promoter_id,background_id,fileformat_id,file
0,10690,1,2024-03-26,1,2024-03-26 14:28:43.825628+00:00,4079,4,6,5,promotersetsig/10690.csv.gz
1,10694,1,2024-03-26,1,2024-03-26 14:28:44.739775+00:00,4083,4,6,5,promotersetsig/10694.csv.gz
2,10754,1,2024-03-26,1,2024-03-26 14:29:01.837335+00:00,4143,4,6,5,promotersetsig/10754.csv.gz
3,10929,1,2024-03-26,1,2024-03-26 14:29:45.379790+00:00,4318,4,6,5,promotersetsig/10929.csv.gz
4,10939,1,2024-03-26,1,2024-03-26 14:29:47.853980+00:00,4327,4,6,5,promotersetsig/10939.csv.gz


In [54]:
# but now the data slot is a dictionary where the `id` are keys and the values
# are the files parsed into pandas dataframes
result.get("data").get("10690")

,name,chr,start,end,strand,experiment_hops,background_hops,background_total_hops,experiment_total_hops,callingcards_enrichment,poisson_pval,hypergeometric_pval
0,1,chrI,0,335,+,0,2,103922,8579,0.0,0.305876,1.0
1,2,chrI,0,538,+,0,4,103922,8579,0.0,0.411518,1.0
2,3,chrI,2169,2480,-,0,1,103922,8579,0.0,0.246143,1.0
3,4,chrI,2169,2480,+,0,1,103922,8579,0.0,0.246143,1.0
4,5,chrI,9017,9717,-,0,11,103922,8579,0.0,0.669806,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6703,7118,chrM,66174,66874,-,0,0,103922,8579,0.0,0.181269,1.0
6704,7132,chrM,74513,75213,-,0,0,103922,8579,0.0,0.181269,1.0
6705,7133,chrM,75984,76684,-,0,0,103922,8579,0.0,0.181269,1.0
6706,7137,chrM,80022,80722,-,0,0,103922,8579,0.0,0.181269,1.0


Parameters can be removed one by one

In [55]:
print(pss_api.params)

regulator_symbol: HAP5, workflow: nf_core_callingcards_dev, data_usable: pass


In [56]:
pss_api.pop_params('data_usable')

print(pss_api.params)

regulator_symbol: HAP5, workflow: nf_core_callingcards_dev


or cleared entirely

In [57]:
pss_api.pop_params(None)

pss_api.params == {}

True

## Rank Response

The rank response endpoint is slightly different than the others.

1. It is an additional action of the promotersetsig table, and so does not have
   a 'records_only' option. Data files will always be retrieved/returned.

1. It currently requires that the parameters `promotersetsig_id` and `expression_id`
   be set.

1. It requires that celery workers are available. On the cluster, this means
    submitting additional jobs. At the time of writing, this hasn't yet been automated.

1. The `id` is the concatenation of the `promotersetsig_id`_`expression_id`

In [58]:
rr_api = RankResponseAPI()

rr_api.push_params({"promotersetsig_id": "8783",
                    "expression_id": "4563"})

result = await rr_api.read()

In [59]:
result.get("metadata")

,id,promotersetsig_id,n_responsive,total_expression_genes,filename,expression_id
0,8783_4563,8783,1074,6175,8783_4563.csv.gz,4563


In [61]:
result.get("data").get("8783_4563")

,rank_bin,n_responsive_in_rank,random,n_successes,response_ratio,pvalue,ci_lower,ci_upper
0,5,2,0.173913,2,0.400000,0.210341,0.052745,0.853367
1,10,1,0.173913,3,0.300000,0.393247,0.066740,0.652453
2,15,1,0.173913,4,0.266667,0.313515,0.077872,0.551003
3,20,0,0.173913,4,0.200000,0.767009,0.057334,0.436614
4,25,1,0.173913,5,0.200000,0.790608,0.068311,0.407037
5,30,2,0.173913,7,0.233333,0.344051,0.099338,0.422837
6,35,0,0.173913,7,0.200000,0.656334,0.084406,0.369379
7,40,1,0.173913,8,0.200000,0.675722,0.090522,0.356478
8,45,0,0.173913,8,0.177778,1.000000,0.080018,0.320534
9,50,0,0.173913,8,0.160000,1.000000,0.071701,0.291126


## Caveats

1. I have written the scripts to automatically check the redis queue for work and to 
   both launch celery worker nodes, and kill them when they are finished. But, though
   they work if I run them manually, they have not worked when scheduled through a
   cronjob. I'll work with Brian and Eric next week to figure out why.

1. I haven't tested each of the endpoint APIs individually. Help is welcome.